In [1]:
import numpy as np

import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
from ray.rllib.models.torch.recurrent_net import DeveloperAPI, RecurrentNetwork

In [3]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [4]:
from ray.rllib.policy.rnn_sequencing import add_time_dimension

class MyModelClass(RecurrentNetwork, nn.Module): # RecurrentNetwork
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        self.time_major = self.model_config.get("_time_major", False)
        self.features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.gru_hidden_dim = 64
        self.gru = nn.GRU(self.features_dim, self.gru_hidden_dim, batch_first=not self.time_major)
        
        self.action_head = nn.Linear(self.gru_hidden_dim, action_space.n)
        self.value_head = nn.Linear(self.gru_hidden_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        
        if isinstance(seq_lens, np.ndarray):
            seq_lens = torch.Tensor(seq_lens).int()
        max_seq_len = features.shape[0] // seq_lens.shape[0]    
        inputs = add_time_dimension(
            features,
            max_seq_len=max_seq_len,
            framework="torch",
            time_major=self.time_major,
        )
        #print('features:', features, 'shape:', features.shape)
        #print('inputs:', inputs, 'shape:', inputs.shape)
        #print('hidden state:', state[0], 'shape:', state[0].shape)
        
        h = state[0].permute(1, 0, 2)
        output, new_h = self.gru(inputs, h)
        new_state = [new_h.permute(1, 0, 2)]
        
        output = output.reshape(-1, self.gru_hidden_dim)
        
        action = self.action_head(output)
        self.last_value = self.value_head(output).squeeze(1)
        return action, new_state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value
    
    @override(TorchModelV2)
    def get_initial_state(self):
        #print('i was here')
        return [torch.zeros(1, self.gru_hidden_dim)]
    
    """@override(RecurrentNetwork)
    def forward_rnn(self, input_dict, state, seq_lens):
        x = nn.functional.relu(self.fc1(input_dict["obs_flat"].float()))
        h_in = state[0].reshape(-1, self.rnn_hidden_dim)
        h = self.rnn(x, h_in)
        q = self.fc2(h)
        self._cur_value = self.value_branch(h).squeeze(1)
        return q, [h]"""

In [5]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C17']))
    env = PovOnlyWrapper(env)
    env = IgluActionWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C17 pretrained (AnnaCNN) + GRU",
                      "notes": "camera noop removed from actions"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc
PPO_my_env_2f3c2_00000,PENDING,


2021-10-01 10:20:22,097	INFO wandb.py:170 -- Already logged into W&B.
2021-10-01 10:20:22,110	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=9363) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=9363)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=9363) 2021-10-01 10:20:26,161	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=9363) 2021-10-01 10:20:26,161	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=9364) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=9364)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=9363) 2021-10-01 10:20:32,889	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=9364) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=9364)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=9364) /root/miniconda/envs/py37/lib/python3.7/site-pac

Result for PPO_my_env_2f3c2_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-10-01_10-21-39
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: ba18be64619d4801b3ca85757cadc21e
  hostname: linar-B360M-D2V
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 2.703709946738349
          entropy_coeff: 0.009999999999999998
          kl: 0.00965274962776467
          policy_loss: -0.0923098218627274
          total_loss: -0.11293265991824025
          vf_explained_var: 0.03169650956988335
          vf_loss: 0.0044837128991882
    num_agent_steps_sampled: 1000
    num_agent_steps_trained: 1000
    num_steps_sampled: 1000
    num_steps_trained: 1000
  iterations_since_restore: 1
  node_ip: 192.168.1.100
  num_

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_2f3c2_00000,RUNNING,192.168.1.100:9363,46,631.139,46000,-0.152174,2,-3,997.413


Result for PPO_my_env_2f3c2_00000:
  agent_timesteps_total: 47000
  custom_metrics: {}
  date: 2021-10-01_10-31-18
  done: false
  episode_len_mean: 997.468085106383
  episode_media: {}
  episode_reward_max: 2.0
  episode_reward_mean: -0.19148936170212766
  episode_reward_min: -3.0
  episodes_this_iter: 1
  episodes_total: 47
  experiment_id: ba18be64619d4801b3ca85757cadc21e
  hostname: linar-B360M-D2V
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 2.6724461767408583
          entropy_coeff: 0.009999999999999998
          kl: 0.009982950807639071
          policy_loss: -0.01870488367146916
          total_loss: 0.02716317938433753
          vf_explained_var: 0.21686363220214844
          vf_loss: 0.07059593113760153
    num_agent_steps_sampled: 47000
    num_agent_steps_trained: 47000
    num_steps_sampled: 47000
    num_steps_trained: 47000
  iterations_since_resto

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_2f3c2_00000,RUNNING,192.168.1.100:9363,47,644.34,47000,-0.191489,2,-3,997.468


Result for PPO_my_env_2f3c2_00000:
  agent_timesteps_total: 48000
  custom_metrics: {}
  date: 2021-10-01_10-31-30
  done: false
  episode_len_mean: 997.5208333333334
  episode_media: {}
  episode_reward_max: 2.0
  episode_reward_mean: -0.1875
  episode_reward_min: -3.0
  episodes_this_iter: 1
  episodes_total: 48
  experiment_id: ba18be64619d4801b3ca85757cadc21e
  hostname: linar-B360M-D2V
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 2.655480718612671
          entropy_coeff: 0.009999999999999998
          kl: 0.008668941282091749
          policy_loss: 0.03999067787081003
          total_loss: 0.01598631539899442
          vf_explained_var: -0.8452224135398865
          vf_loss: 0.0008166570783942007
    num_agent_steps_sampled: 48000
    num_agent_steps_trained: 48000
    num_steps_sampled: 48000
    num_steps_trained: 48000
  iterations_since_restore: 48
  nod

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_2f3c2_00000,RUNNING,192.168.1.100:9363,48,655.991,48000,-0.1875,2,-3,997.521
